# Make custom ARIBA database

For the *N. gonorrhoeae* data of the manuscript, we were interested in particular sequences and SNPs. This means using custom reference data as opposed to one of the public reference sets, the use of which is described [in the ARIBA wiki page](https://github.com/sanger-pathogens/ariba/wiki/Task:-getref).

The starting point is a multiple alignment of reference sequences.

To do ...